In [ ]:
import sys, os
from dotenv import load_dotenv
load_dotenv()

sys.path.append(os.environ.get("path_to_auem"))
from torch.utils import tensorboard as tb
from auem.data.datasets import DCASE2020Task1a
from torch.utils.data import DataLoader
from auem.models.simplenn import SimpleNN
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import torch
from torch.nn.functional import softmax
from torch.nn import Transformer, Linear, Module


batch_size = 32
device = "cuda:0"

In [ ]:
ds = DCASE2020Task1a(split_df_path="/home/const/Projects/data/dcase2020/task1a/TAU-urban-acoustic-scenes-2020-mobile-development/evaluation_setup/fold1_train.csv",
                     metadata_df_path="/home/const/Projects/data/dcase2020/task1a/TAU-urban-acoustic-scenes-2020-mobile-development/meta.csv",
                     data_root="/home/const/Projects/data/dcase2020/task1a/TAU-urban-acoustic-scenes-2020-mobile-development/")

train_size = int(0.8 * len(ds))
valid_size = len(ds) - train_size
train_ds, valid_ds = torch.utils.data.random_split(ds, [train_size, valid_size])

dl_train = DataLoader(train_ds, batch_size=batch_size)
dl_valid = DataLoader(valid_ds, batch_size=batch_size)
# model = SimpleNN(282368).to(device)

In [ ]:
# model = Transformer(d_model=128).to(device)
class TC(Module):
    def __init__(self):
        super(TC, self).__init__() 
        self.transformer = Transformer(d_model=128, 
                                       nhead=8,
                                       num_encoder_layers=1,
                                       num_decoder_layers=1,
                                       dim_feedforward=2048,
                                       dropout=.2,
                                       activation='gelu')
        self.linear = Linear(128, 10)

    def forward(self, x):
        out = self.transformer(x, x)
        out = self.linear(out[:, 0])
        return softmax(out)

model = TC().to(device)

In [ ]:
optimizer = Adam(model.parameters())
criterion = CrossEntropyLoss()

In [ ]:
x = iter(dl_train).next()['X'].permute(0,1,3,2).squeeze().to('cuda')

In [ ]:

x.shape

In [ ]:
writer = tb.SummaryWriter()
writer.add_graph(model, x, verbose=True)

In [ ]:
writer.flush()
# writer.close()

In [ ]:
# src = torch.rand((10, 32, 512))
# tgt = torch.rand((20, 32, 512))
# out = model(x, x)

In [ ]:
pos_enc = torch.range(0, 2205).unsqueeze(0)
for epoch in range(100):
    print(f"epoch {epoch}")
    model.train()
#     for batch_num, batch in tqdm(enumerate(dl_train), total=ceil(len(dl_train)/batch_size), desc="Batch"):
    for batch_num, batch in enumerate(dl_train):
        X, y = batch["X"].permute(0,1,3,2).squeeze().to(device), batch["label"].to(device)
        X[:,:, 0] = pos_enc.repeat(X.shape[0], 1)
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        writer.add_scalar("loss2/TC", loss.item(), global_step=batch_num)
writer.add_embedding(output.to("cpu"), metadata=y.tolist(), global_step=batch_num)#, label_img=batch['X'])
        # writer.add_image('epoch/5', batch['X'], global_step=batch_num, dataformats='NCHW')

In [ ]:
pos_enc = torch.range(0, 2205).unsqueeze(0).repeat(batch_size, 1)
pos_enc.shape

In [ ]:
# x = iter(dl_train).next()['X'].to('cuda')
%pylab inline
imshow(pos_enc, aspect='auto', interpolation='none')

In [ ]:
writer.flush()

In [ ]:
writer.close()